In [ ]:
#cum4
params = [
    {'max_depth':9, 
     'eta':0.01, # 'objective':'reg:linear',
     'eval_metric':'rmse', #'maximize': False,
     'colsample_bytree':0.6, #7
     'subsample':0.8,  #8
     'min_child_weight': 4.0,
     'nthread':32,
     'silent': True
    },
    {'max_depth':8, 
     'eta':0.02, # 'objective':'reg:linear',
     'eval_metric':'rmse', #'maximize': False,
     'colsample_bytree':0.3, #7
     'subsample':0.8,  #8
     'min_child_weight': 4.0,
     'nthread':32,
     'silent': True
    },
    {'max_depth':10, 
     'eta':0.01, # 'objective':'reg:linear',
     'eval_metric':'rmse', #'maximize': False,
     'colsample_bytree':0.6, #7
     'subsample':0.8,  #8
     'min_child_weight': 4.0,
     'nthread':32,
     'silent': True
    },
    {'max_depth':8, 
     'eta':0.01, # 'objective':'reg:linear',
     'eval_metric':'rmse', #'maximize': False,
     'colsample_bytree':0.5, #7
     'subsample':0.9,  #8
     'min_child_weight': 6.0,
     'nthread':32,
     'silent': True
    }
]
num_rounds = [1000, 580, 680, 1000]

In [1]:
import time
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn import linear_model
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestRegressor
#from sklearn import pipeline, model_selection
from sklearn import pipeline, grid_search
#from sklearn.feature_extraction import DictVectorizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion
from sklearn.decomposition import TruncatedSVD
#from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_squared_error, make_scorer

import re

import random
random.seed(2016)

dtrain = xgb.DMatrix("train.buffer")
dtest = xgb.DMatrix("test.buffer")
evallist  = [(dtrain,'train')]


loc = '/home/ec2-user/data/hd/features/%s'
a_o = np.load(loc % 'train_data.npy')
b_o = np.load(loc % 'test_data.npy')
a_brand = np.load(loc % 'features_brand_01_train.npy')
b_brand = np.load(loc % 'features_brand_01_test.npy')
a_other = np.load(loc % 'FEATURES_1d_TRAIN.npy')
b_other = np.load(loc % 'FEATURES_1d_TEST.npy')
a_word_feat = np.load(loc % 'SPECIAL_WORDS_FEAT_TRAIN.npy')
b_word_feat = np.load(loc % 'SPECIAL_WORDS_FEAT_TEST.npy')
#FINAL GENERATE
aaa = pd.read_pickle(loc % 'WOQTAL_TRAIN_ALL')
bbb = pd.read_pickle(loc % 'WOQTAL_TEST_ALL')
aa = aaa.drop('relevance', axis=1).values[:,[0,3]]
bb = bbb.drop('relevance', axis=1).values[:,[0,3]]

a = np.hstack((a_o, a_brand, a_other, a_word_feat, aa))
b = np.hstack((b_o, b_brand, b_other, b_word_feat, bb))

ggX_train = xgb.DMatrix(data=a, label=dtrain.get_label())
ggX_test = xgb.DMatrix(data=b)
idx_train = pd.read_pickle(loc % 'LABELS_TRAIN.df')
idx_test = pd.read_pickle(loc % 'LABELS_TEST.df')


params = [
    {'max_depth':9, 
     'eta':0.01, # 'objective':'reg:linear',
     'eval_metric':'rmse', #'maximize': False,
     'colsample_bytree':0.7, #7
     'subsample':0.9,  #8
     'min_child_weight': 4.0,
     'nthread':32,
     'silent': True
    },
    {'max_depth':7, 
     'eta':0.03, # 'objective':'reg:linear',
     'eval_metric':'rmse', #'maximize': False,
     'colsample_bytree':0.7, #7
     'subsample':0.9,  #8
     'min_child_weight': 4.0,
     'nthread':32,
     'silent': True
    },
    {'max_depth':7, 
     'eta':0.01, # 'objective':'reg:linear',
     'eval_metric':'rmse', #'maximize': False,
     'colsample_bytree':0.7, #7
     'subsample':0.9,  #8
     'min_child_weight': 4.0,
     'nthread':32,
     'silent': True
    },
    {'max_depth':8, 
     'eta':0.01, # 'objective':'reg:linear',
     'eval_metric':'rmse', #'maximize': False,
     'colsample_bytree':0.7, #7
     'subsample':0.9,  #8
     'min_child_weight': 6.0,
     'nthread':32,
     'silent': True
    }
]
num_rounds = [1000, 450, 700, 1000]

def next_param():
    return np.random.randint(0,len(params))

running_total = pd.DataFrame({'A' : []})
for i in range(20000):
    print ("Starting run: %d" % i)
    
    r = next_param() 
    param = params[r]
    param['seed'] = i*137 + 110
    num_round = num_rounds[r]
    print ("Num rounds: %d" % num_round)
    
    bst = xgb.train( param, ggX_train, num_round, [(gX_test,'test')], verbose_eval=200)
    y_pred = bst.predict(ggX_test)    
    y_pred_bounded = np.minimum(np.maximum(y_pred, 1.0), 3.0)
    idx_test['relevance'] = y_pred_bounded
    idx_test.to_csv('submission_xgboost_many5_%05d.csv' % i)
    
    if not running_total.empty:
        running_total = running_total + idx_test
    else:
        running_total = idx_test
        
    y_pred_bounded = np.minimum(np.maximum(running_total['relevance']/(i+1), 1.0), 3.0)
    idx_test['relevance'] = y_pred_bounded
    idx_test.to_csv('submission_xgboost_cum5_%05d.csv' % i)
    
            

In [43]:
np.random.randint(0,2)

1

In [8]:
np.random.randint(750, 950)

934